In [21]:
%load_ext autoreload
%autoreload 2

from src.jazz_ai import JazzAI
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
import src.load_data as load_data
from IPython.display import display
import pandas as pd

# load in the data
training_data = load_data.data_pipeline()
mdl = JazzAI(RandomForestClassifier)
mdl.fit(training_data)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,actual,pred,#4,1,2,3,4,5,6,7,b2,b3,b6,b7
4,4,3,0.000000,0.118436,0.000000,0.610343,0.138148,0.133073,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,1,#4,0.587548,0.347452,0.010000,0.025000,0.000000,0.026000,0.000000,0.000000,0.000000,0.004000,0.000000,0.000000
6,1,1,0.000000,0.624798,0.051149,0.033750,0.013411,0.050435,0.073750,0.050000,0.032019,0.038188,0.010000,0.022500
7,5,5,0.001667,0.017417,0.002500,0.000000,0.028917,0.495167,0.000000,0.404833,0.000000,0.044500,0.000000,0.005000
8,2,2,0.010000,0.046972,0.642000,0.001111,0.040250,0.040667,0.030389,0.001667,0.000000,0.016667,0.018333,0.151944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200803,7,5,0.000000,0.139804,0.000000,0.078263,0.088474,0.316860,0.077909,0.220063,0.000000,0.078627,0.000000,0.000000
200804,5,5,0.000000,0.136979,0.098892,0.000000,0.000000,0.387241,0.376888,0.000000,0.000000,0.000000,0.000000,0.000000
200805,4,6,0.000000,0.090570,0.072221,0.105548,0.068831,0.027950,0.518410,0.078880,0.000000,0.000000,0.000000,0.037592
200806,5,3,0.104975,0.000000,0.000000,0.619985,0.000000,0.183670,0.000000,0.000000,0.000000,0.091370,0.000000,0.000000


In [22]:
import numpy as np

from src.feature_matrix import create_feature_matrix

solo = training_data.query('melid==93')[['onset', 'octave', 'duration', 'chord', 'chord_root', 'melody_note', 'chord_3rd', 'chord_5th', 'chord_7th', 'chord_root_num']].reset_index(drop=True)
struct_df = solo[['chord_3rd', 'chord_5th', 'chord_7th', 'chord_root_num']].iloc[4:,]
init_notes = ["b3", "3", "6", "b7"]

generated_notes = mdl.generate_solo(init_notes, struct_df)
# generated_notes.index = struct_df.index
generated_notes

,chord_3rd,chord_5th,chord_7th,chord_root_num,generated_notes
0,NaN,NaN,NaN,NaN,b3
1,NaN,NaN,NaN,NaN,3
2,NaN,NaN,NaN,NaN,6
3,NaN,NaN,NaN,NaN,b7
4,0.0,1.0,0.0,8.0,5
...,...,...,...,...,...
450,1.0,1.0,0.0,1.0,b3
451,1.0,1.0,0.0,1.0,6
452,1.0,1.0,0.0,1.0,4
453,1.0,1.0,0.0,1.0,1


In [25]:
from src.music_theory import calculate_note_from_interval
import pandas as pd

solo_melody_notes = pd.merge(solo, generated_notes, left_index=True, right_index=True, how='outer')[['melody_note', 'onset', 'octave', 'duration', 'chord', 'chord_root', 'generated_notes']]
solo_melody_notes = calculate_note_from_interval(solo_melody_notes, 'chord_root', 'generated_notes', 'generated_note_names')
solo_melody_notes = solo_melody_notes.drop(['chord_root', 'generated_notes'], axis=1)
solo_melody_notes = solo_melody_notes.dropna(axis=0)

solo_melody_notes['octave'] = 4
solo_melody_notes

,melody_note,onset,octave,duration,chord,generated_note_names
0,G#,3.178231,4,0.182857,Fj7,G#
1,A,3.361088,4,0.128980,Fj7,A
2,D,3.612245,4,0.128980,Fj7,D
3,F,3.751020,4,0.278299,G-7,F
4,G,4.032653,4,0.101587,G-7,D
...,...,...,...,...,...,...
449,F,96.828571,4,0.190000,G-7,F#
450,A#,97.026531,4,0.192086,C7,D#
451,G,97.220408,4,0.092880,C7,A
452,E,97.344898,4,0.098707,C7,F


In [26]:
from src.generate_sound import generate_solo_sounds


bass_note_data = load_data.bass_note_pipeline()
solo_bass_notes = bass_note_data.query(f'melid==93')

W = generate_solo_sounds(solo_bass_notes, solo_melody_notes, melody_note_col_name='generated_note_names', bass_note_duration=0.2, snare_duration=0.1, snare_beats=[2, 4])
W.play()


Note: falling back to playback by exporting to temporary file then using system default player. It is recommended to install one of the supported I/O libraries for more flexible playback, for example simpleaudio, pygame or playsound (see docs for more). 
Temp file created at: /Users/arvinlim/Projects/JazzAI\gensound_temp/gensound_temp_2023_01_10__22_02_55_hgtgpovx.wav
